# This snippet divides the default display channel and the rest channels

###### The default channel will be used to display a clear image during segmentation. It will be an image file in numpy array. The rest channels are channels that the user will not see during segmentation. The rest channels file will be a list containing images in numpy array. Both the default and the rest channels that are created here are in pickle format. When the pickle is loaded, the default (np.array) or restchannels (list with np arrays) can be accessed. <br>

### select_unprocessed_czi_directory() :
####    Will prompt the user to select a folder with all czi files. The 'latest_directory' will be a string with the path of that folder. See 'select_default_channel' too before you proceed.
<br>

### select_save_dir() :
#### Will prompt the user to select a folder where the transformed files will be saved. The 'save_dir' will be a string with the path of that folder.
<br>

### select_default_channel():
#### Will prompt the user to select the channel that wants on display during segmentation. Only one channel will be visible. A random image (randomly_picked_img) is selected outside of this function. If the number of channels of this image (which is indicated in the last element of the SOMEIMAGEHERE.shape) is larger than the users input, then an error will occur. Of note, the order of channels might have been reordered, thereby leading to a selection of a wrong channel for display by the user. Hence, it is recommended that the user tests what number corresponds to what channel by selecting a directory with only one image. Then, (s)he can open the segmentation_pipeline.ipynb and test if the inserted number corresponds to the right number. 
<br>

#### Information about save :
##### The files will be saved in this form : SAMENAMEHERE-default.pkl and SAMENAMEHERE-restchannels.pkl for each czi file with SAMENAME

In [ ]:
import cv2
import pickle
import os
import sys
import timeit
import numpy as np
import czifile
from shapely.geometry import Polygon
from itertools import cycle, islice 
from tkinter import *
from tkinter import filedialog
from tkinter.filedialog import askdirectory
from tkinter import messagebox
from os import listdir
from os.path import isfile, join, exists

def select_unprocessed_czi_directory ():
    def select_specific_directory():
        global latest_directory
        latest_directory = filedialog.askdirectory(title='Select czi folder')
        root.destroy()
    root = Tk()
    root.title ('Transform Images')
    root.resizable(True,True)
    root.geometry('300x200')
    DirButton = Button(root,text='Locate your czi image folder', command=select_specific_directory).place(x=75,y=80)
    root.mainloop()

def select_save_dir ():
    def select_spec_save_dir():
        global save_dir
        save_dir = filedialog.askdirectory(title='Where to save')
        sroot.destroy()
    sroot = Tk()
    sroot.title ('Channels folder')
    sroot.resizable(True,True)
    sroot.geometry('300x200')
    DirButton = Button(sroot,text='Select folder to save pkl channels ', command=select_spec_save_dir).place(x=70,y=80)
    sroot.mainloop()
    
def select_default_channel():
    def get_channel_entry ():
        global default_channel
        try :
            default_channel = int(select_entry.get()) - 1
            if default_channel+1 > example_n_of_channels :
                messagebox.showerror(title='Exceeded number of channels',message=f'You have selected channel {default_channel+1} but based on a random file ({images_in_dir[0]}) your images have {example_n_of_channels}')
                return
        except :
            messagebox.showerror(message='There is something wrong with your input. Please type a number.')
        else :
            selectroot.destroy()
    
    selectroot = Tk()
    selectroot.title('Display channel selection')
    select_label = Label(selectroot, text="Type the number of your default channel for display")
    select_label.pack( side = LEFT)
    select_entry = Entry(selectroot, bd =5)
    select_entry.pack(side = RIGHT)
    select_button = Button(selectroot, text='OK', command = get_channel_entry).pack(side = RIGHT)
    selectroot.mainloop()

select_unprocessed_czi_directory()    
select_save_dir()
images_in_dir         = [img for img in listdir(latest_directory) if isfile(join(latest_directory, img))]
randomly_picked_img   = czifile.imread(join(latest_directory,images_in_dir[0])) 
randomly_picked_img   = np.squeeze(randomly_picked_img)
randomly_picked_img   = np.transpose(randomly_picked_img, (1,2,0))
example_n_of_channels = randomly_picked_img.shape[-1]
select_default_channel()



for img in images_in_dir :
    path = f'{latest_directory}/{img}'
    try :
        eikona = czifile.imread(path)                                                           # 4 secs
    except ValueError:
        notcizroot = Tk()
        messagebox.showerror(title='Wrong file extension', message='Only czi files are allowed in image folder. This program will terminate')
        notcizroot.destroy()
        sys.exit(0)
    
    # Squeeze the image to remove the first and last dimensions
    eikona = np.squeeze(eikona)                                                    # we want to convert from 5 dimensions to 3. So we reduce the dimensions with a value of one (beginning and end)

    # Transpose the image to rearrange the dimensions                              # the channel dimension goes from start to end to fit with opencv
    eikona = np.transpose(eikona, (1, 2, 0))                  

    # # Convert the CZI image to BGR color format for OpenCV
    eikona = cv2.cvtColor(eikona, cv2.COLOR_RGB2BGR)                              
    n_of_channels = eikona.shape[-1]
    
    with open(f'{save_dir}/{img}-restchannels.pkl', 'wb') as writa:                                     
        Channels = []
        for channel_index in range(n_of_channels): 
            if channel_index == default_channel :
                continue
            else :
                single_channel = eikona[:,:,channel_index]
                Channels.append(single_channel)
        pickle.dump(Channels,writa)
    
    with open(f'{save_dir}/{img}-default.pkl', 'wb') as writable:                                     
        default_ch = eikona[:,:,default_channel]
        pickle.dump(default_ch,writable)
   


# 